In [2]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/kir/worker_notebooks/imports.ipynb")
execute_notebook("/cellar/users/ramarty/Projects/kir/worker_notebooks/clinical.ipynb")
execute_notebook("/cellar/users/ramarty/Projects/kir/worker_notebooks/typing.ipynb")

Populating the interactive namespace from numpy and matplotlib


"\ndef plot_cutoffs(gene, w):\n    kernel = sp.gaussian_kde(w[gene])\n    max_value = w[gene].max()\n    a = kernel(np.arange(0, max_value, max_value/float(100)))\n    coverage_values = np.arange(0, max_value, max_value/float(100))\n    plt.scatter(coverage_values, a)\n    for l in coverage_values[np.r_[True, a[1:] < a[:-1]] & np.r_[a[:-1] < a[1:], True]]:\n        plt.axvline(x=l, ymin=0, ymax=1, linewidth=2, color='k')\n    plt.title(gene)\n    plt.show()\n    plt.clf()\n"

In [3]:
df = pd.read_csv('/cellar/users/ramarty/Data/kir/biological/all_the_good_stuff.csv', index_col=0)
df = df[~(df.tissue.isin(['LIHC', 'DLBC', 'ACC', 'BRCA', 'UCS', 'SKCM']))]

In [4]:
df.head()

,Tumor_Sample_Barcode,num_neoantigens5,num_neoantigens10,total_presented_peptides,mutation_count,max_neoantigen,num_neoantigens1,KIR2DP1,KIR2DS4,KIR2DS5,...,A1,A2,B1,B2,C1,C2,activating,inhibitory,ratio,capture_kit
0,TCGA-05-4244,65,19,863,200,31,130,1,1,1,...,A*01,A*02,B*41,B*52,C*12,C*17,9,11,0.818182,Agilent_50
1,TCGA-05-4250,99,25,1465,349,29,244,2,2,0,...,A*01,A*11,B*45,B*57,C*06,C*06,4,12,0.333333,Agilent_50
2,TCGA-05-4384,39,14,586,136,53,78,2,2,0,...,A*02,A*23,B*37,B*39,C*06,C*12,4,12,0.333333,Agilent_50
3,TCGA-05-4390,175,62,2476,555,34,369,2,1,1,...,A*01,A*02,B*07,B*08,C*07,C*07,6,13,0.461538,Agilent_50
4,TCGA-05-4396,135,41,2015,488,25,330,2,0,1,...,A*01,A*02,B*08,B*40,C*03,C*07,5,12,0.416667,Agilent_50


In [5]:
df.tissue.value_counts()

LUAD    378
HNSC    367
LGG     240
THCA    215
BLCA    198
LUSC     91
Name: tissue, dtype: int64

In [6]:
df.columns

Index([u'Tumor_Sample_Barcode', u'num_neoantigens5', u'num_neoantigens10',
       u'total_presented_peptides', u'mutation_count', u'max_neoantigen',
       u'num_neoantigens1', u'KIR2DP1', u'KIR2DS4', u'KIR2DS5', u'KIR2DS2',
       u'KIR2DS3', u'KIR2DS1', u'KIR2DL3', u'KIR2DL2', u'KIR2DL1', u'KIR2DL4',
       u'KIR3DS1', u'KIR3DL1', u'KIR3DL2', u'KIR3DL3', u'KIR2DL5A', u'KIR3DP1',
       u'Sample', u'barcode', u'age_at_diagnosis', u'days_to_death',
       u'days_to_last_followup', u'ethnicity', u'gender', u'race', u'tissue',
       u'vital_status', u'Bcells', u'Monocytes', u'Neutrophils', u'NKcells',
       u'Tcells', u'unkCells', u'A1', u'A2', u'B1', u'B2', u'C1', u'C2',
       u'activating', u'inhibitory', u'ratio', u'capture_kit'],
      dtype='object')

In [7]:
neoantigen_features = ['num_neoantigens10',
       'total_presented_peptides', 'mutation_count']

###Tissue agnostic

Activating/inhibitory

In [8]:
for neo in neoantigen_features:
    print neo, sp.spearmanr(df[neo], df.inhibitory)

num_neoantigens10 SpearmanrResult(correlation=0.023047114213981729, pvalue=0.37416264810448641)
total_presented_peptides SpearmanrResult(correlation=0.014537412173586412, pvalue=0.57512356206976545)
mutation_count SpearmanrResult(correlation=0.014727242604054745, pvalue=0.57014230186860981)


In [9]:
for neo in neoantigen_features:
    print neo, sp.spearmanr(df[neo], df.activating)

num_neoantigens10 SpearmanrResult(correlation=-0.063620917186093986, pvalue=0.014072706863007651)
total_presented_peptides SpearmanrResult(correlation=-0.062074292406916066, pvalue=0.016593437753802506)
mutation_count SpearmanrResult(correlation=-0.056421007700279809, pvalue=0.029476459910841468)


Gene specific

In [10]:
genes = ['KIR3DL3', 'KIR3DP1', 'KIR2DL4', 'KIR3DL2', 'KIR2DS2', 'KIR2DS3', 'KIR2DP1', 
         'KIR2DL1', 'KIR3DS1', 'KIR2DS5', 'KIR2DS4']

In [16]:
for gene in genes:
    for neo in neoantigen_features:
        print gene, neo, sp.spearmanr(df[neo], df[gene])
    print ' '

KIR3DL3 num_neoantigens10 SpearmanrResult(correlation=0.00030640991031757707, pvalue=0.99057426454384179)
KIR3DL3 total_presented_peptides SpearmanrResult(correlation=0.0044819087514771398, pvalue=0.86280741026145558)
KIR3DL3 mutation_count SpearmanrResult(correlation=0.0021808489173523074, pvalue=0.93299038432030712)
 
KIR3DP1 num_neoantigens10 SpearmanrResult(correlation=-0.022812724303365239, pvalue=0.37904210185877418)
KIR3DP1 total_presented_peptides SpearmanrResult(correlation=-0.022431054571030361, pvalue=0.387070822241526)
KIR3DP1 mutation_count SpearmanrResult(correlation=-0.023167730964503825, pvalue=0.37166688327093822)
 
KIR2DL4 num_neoantigens10 SpearmanrResult(correlation=-0.028443226903616298, pvalue=0.27270479598787084)
KIR2DL4 total_presented_peptides SpearmanrResult(correlation=-0.027059248723065376, pvalue=0.29673315643703846)
KIR2DL4 mutation_count SpearmanrResult(correlation=-0.027261839264359208, pvalue=0.29313007446501355)
 
KIR3DL2 num_neoantigens10 SpearmanrRes

###Tissue dependent

Activating/inhibitory

In [17]:
for tissue in df.tissue.unique():
    print tissue
    t = df.tissue == tissue
    for neo in neoantigen_features:
        print neo, sp.spearmanr(df[t][neo], df[t].inhibitory)
    print ' '

LUAD
num_neoantigens10 SpearmanrResult(correlation=0.014595388838234023, pvalue=0.77729792233659523)
total_presented_peptides SpearmanrResult(correlation=0.013147350153889771, pvalue=0.79889445867401909)
mutation_count SpearmanrResult(correlation=0.014163485411261452, pvalue=0.78372238326048094)
 
LUSC
num_neoantigens10 SpearmanrResult(correlation=-0.08107775254970416, pvalue=0.44486745187022902)
total_presented_peptides SpearmanrResult(correlation=-0.20738774246585234, pvalue=0.048551886042005943)
mutation_count SpearmanrResult(correlation=-0.22452389601860745, pvalue=0.032386214063309306)
 
HNSC
num_neoantigens10 SpearmanrResult(correlation=0.040109989576911288, pvalue=0.44362636519730392)
total_presented_peptides SpearmanrResult(correlation=0.0058458692159323051, pvalue=0.91113291396439822)
mutation_count SpearmanrResult(correlation=0.0031459748239349222, pvalue=0.95210564924638685)
 
THCA
num_neoantigens10 SpearmanrResult(correlation=-0.0095952583022967559, pvalue=0.888757064539054

In [18]:
for tissue in df.tissue.unique():
    print tissue
    t = df.tissue == tissue
    for neo in neoantigen_features:
        print neo, sp.spearmanr(df[t][neo], df[t].activating)
    print ' '

LUAD
num_neoantigens10 SpearmanrResult(correlation=0.0022315678707621627, pvalue=0.96550782901499899)
total_presented_peptides SpearmanrResult(correlation=-0.0058260310635034245, pvalue=0.91011237067171824)
mutation_count SpearmanrResult(correlation=-0.0048893107174290528, pvalue=0.92451753039006102)
 
LUSC
num_neoantigens10 SpearmanrResult(correlation=-0.1046517034415126, pvalue=0.3235312993172359)
total_presented_peptides SpearmanrResult(correlation=-0.065182200835014789, pvalue=0.53931033068339196)
mutation_count SpearmanrResult(correlation=-0.080598691487953789, pvalue=0.44757107522641992)
 
HNSC
num_neoantigens10 SpearmanrResult(correlation=-0.059155582691542696, pvalue=0.25831523320778904)
total_presented_peptides SpearmanrResult(correlation=-0.075067259173563092, pvalue=0.15122963494358971)
mutation_count SpearmanrResult(correlation=-0.075735193860440031, pvalue=0.14761467733416347)
 
THCA
num_neoantigens10 SpearmanrResult(correlation=0.02713455902921259, pvalue=0.69238229230198

In [19]:
for tissue in df.tissue.unique():
    print tissue
    t = df.tissue == tissue
    for neo in neoantigen_features:
        for gene in genes:
            print neo, gene, sp.spearmanr(df[t][neo], df[t][gene])
    print ' '

LUAD
num_neoantigens10 KIR3DL3 SpearmanrResult(correlation=-0.08039850838012419, pvalue=0.11864749999830823)
num_neoantigens10 KIR3DP1 SpearmanrResult(correlation=0.026131253846914664, pvalue=0.61253831585362484)
num_neoantigens10 KIR2DL4 SpearmanrResult(correlation=0.087940793113112314, pvalue=0.087745661903752076)
num_neoantigens10 KIR3DL2 SpearmanrResult(correlation=0.018696886049945599, pvalue=0.71710038116184927)
num_neoantigens10 KIR2DS2 SpearmanrResult(correlation=-0.049433288699845229, pvalue=0.33781269868728647)
num_neoantigens10 KIR2DS3 SpearmanrResult(correlation=-0.048233345913174311, pvalue=0.34968418699934822)
num_neoantigens10 KIR2DP1 SpearmanrResult(correlation=-0.0072485830403427529, pvalue=0.88829366102689966)
num_neoantigens10 KIR2DL1 SpearmanrResult(correlation=-0.0077816009038715822, pvalue=0.8801391422274365)
num_neoantigens10 KIR3DS1 SpearmanrResult(correlation=-0.019960278251986145, pvalue=0.69888544214374426)
num_neoantigens10 KIR2DS5 SpearmanrResult(correlatio

### KIR/MHC combinations

In [11]:
hla_a = [x.split(',')[1:3] for x in open('/cellar/users/ramarty/Data/kir/biological/combinations.hla_a.csv').readlines()[1:]]
hla_b = [x.split(',')[1:3] for x in open('/cellar/users/ramarty/Data/kir/biological/combinations.hla_b.csv').readlines()[1:]]
hla_c = [x.split(',')[1:3] for x in open('/cellar/users/ramarty/Data/kir/biological/combinations.hla_c.csv').readlines()[1:]]

In [15]:
predicted_total = 100 # update
total = 0
for gene in genes:
    for hla1, hla2 in hla_a:
        for neo in neoantigen_features:
            h = (df.A1 == hla1)&(df.A2 == hla2)
            k = df[gene] > 1
            if len(df[k&h][neo]) > 5 and len(df[~k&h][neo]) > 5:
                total += 1
                if (sp.ttest_ind(df[k&h][neo], df[~k&h][neo])[1]) < 0.05:
                    print neo, gene, hla1, hla2, sp.ttest_ind(df[k&h][neo], df[~k&h][neo])
print total

total_presented_peptides KIR2DS4 A*02 A*02 Ttest_indResult(statistic=2.095473679693669, pvalue=0.038556853072887781)
mutation_count KIR2DS4 A*02 A*02 Ttest_indResult(statistic=2.0604112924790989, pvalue=0.041855286536953341)
219


In [17]:
predicted_total = 100 # update
total = 0
for gene in genes:
    for hla1, hla2 in hla_b:
        for neo in neoantigen_features:
            h = (df.B1 == hla1)&(df.B2 == hla2)
            k = df[gene] > 1
            if len(df[k&h][neo]) > 5 and len(df[~k&h][neo]) > 5:
                total += 1
                if (sp.ttest_ind(df[k&h][neo], df[~k&h][neo])[1]) < 0.05:
                    print neo, gene, hla1, hla2, sp.ttest_ind(df[k&h][neo], df[~k&h][neo])
print total

135


In [18]:
predicted_total = 100 # update
total = 0
for gene in genes:
    for hla1, hla2 in hla_c:
        for neo in neoantigen_features:
            h = (df.C1 == hla1)&(df.C2 == hla2)
            k = df[gene] > 1
            if len(df[k&h][neo]) > 5 and len(df[~k&h][neo]) > 5:
                total += 1
                if (sp.ttest_ind(df[k&h][neo], df[~k&h][neo])[1]) < 0.05:
                    print neo, gene, hla1, hla2, sp.ttest_ind(df[k&h][neo], df[~k&h][neo])
print total

num_neoantigens10 KIR3DL2 C*07 C*07 Ttest_indResult(statistic=-2.4057434649648255, pvalue=0.017309720856048507)
total_presented_peptides KIR3DL2 C*07 C*07 Ttest_indResult(statistic=-2.3532452568955433, pvalue=0.019857538165528477)
mutation_count KIR3DL2 C*07 C*07 Ttest_indResult(statistic=-2.3467118607305015, pvalue=0.020196691395797217)
num_neoantigens10 KIR3DL2 C*07 C*12 Ttest_indResult(statistic=-2.6646505829320692, pvalue=0.010240062808754342)
total_presented_peptides KIR3DL2 C*07 C*12 Ttest_indResult(statistic=-2.5919630311058834, pvalue=0.012359652205709703)
mutation_count KIR3DL2 C*07 C*12 Ttest_indResult(statistic=-2.5318682058975024, pvalue=0.014406023655204421)
mutation_count KIR2DP1 C*03 C*06 Ttest_indResult(statistic=2.0591647147075123, pvalue=0.045867938636460222)
total_presented_peptides KIR2DP1 C*04 C*06 Ttest_indResult(statistic=-2.2428141793563734, pvalue=0.031964252154628732)
mutation_count KIR2DP1 C*04 C*06 Ttest_indResult(statistic=-2.2497951406182239, pvalue=0.0314